# Do prediction on unseen data



In [1]:
# Requirements for data preprocessing

# Transformers needed for text embedding
!pip install transformers

import os

# copy utils file
if not os.path.isfile('preprocess_utils.py'):
  !gdown --id 1EsKIkfWO2EyInI1fdMJB-KdrE7DybgIr

from preprocess_utils import reuters_util

if not os.path.isfile('keras_model.h5'):
  !gdown --id 1ZxbmSwjIQAt_yMfK1Xxhi5NzQBHPtnjK

     |████████████████████████████████| 3.1 MB 14.5 MB/s 
     |████████████████████████████████| 3.3 MB 35.2 MB/s 
     |████████████████████████████████| 895 kB 37.7 MB/s 
     |████████████████████████████████| 59 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 41.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Downloading...
From: https://drive.google.com/uc?id=1EsKIkfWO2EyInI1fdMJB-KdrE7DybgIr
To: /content/preprocess_utils.py
100% 6.99k/6.99k [00:00<00:00, 15.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZxbmSwjIQAt_yMfK1Xxhi5NzQBHPtnjK
To: /content/keras_model.h5
100% 9.48M/9.48M [00:00<00:00, 44.3MB/s]


In [2]:
from bs4 import BeautifulSoup
from string import punctuation
import re
import numpy as np
import pandas as pd
import torch
import transformers
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_score, recall_score, balanced_accuracy_score
import nltk
import tensorflow as tf
import math
import h5py
import matplotlib.pyplot as plt
from tensorflow.python.framework import ops
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import optimizers
from keras.regularizers import l1

In [3]:
# Create object for the reuters_util class
utils_obj = reuters_util()

In [4]:
# Configure GPU

utils_obj.gpu_config()

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [5]:
# Initialize prebuilt models to do the embeddings

utils_obj.initialize_prebuilt_models()

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
# Upload the test data to the current session and
# Replace test_data_path by absolute path for the test data directory.
# The test files should be in Standard Generalized Markup Language format   

test_data_path = "/content"



## **Warning**: 




It is expected for the topics of each document to be parsed in the SGM file. If the given test data files don't include the topics and want to provided it in a separate file, pass the topics list as as **`y_true`** in the do_prediction function below. 

If the topics are not found from the document and the **`y_true`** variable  is set to false, then the topics are assumed to be of 0 class (not-earn). 

In [ ]:
test_data_embedding = utils_obj.preprocess_data(test_data_path)

In [ ]:
test_data_embedding.shape

In [ ]:
test_data_embedding.head()

In [ ]:
def do_prediction(test_data, classifier, treshold_val=0.5, y_true=False):
  if y_true:
    y_test = y_true
  else:
    y_test, x_test = test_data['topics'], test_data.iloc[:, 0:-1]
  # Prediction results
  y_pred = classifier.predict(x_test)
  y_pred = (y_pred > treshold_val)
  # Create a dataframe for actual vs predicted class
  result_df = pd.DataFrame()
  result_df["actual_class"], result_df["pred_class"] = y_test, y_pred
  return result_df

In [ ]:
def model_scoring(actual_vs_pred_result):
  # calculate precision, recall, auc and balanced_accuracy
  y_true = actual_vs_pred_result["actual_class"]
  y_pred = actual_vs_pred_result["pred_class"]
  recall_0, recall_1 = recall_score(y_true, y_pred, average=None)
  precision_0, precision_1  = precision_score(y_true, y_pred, average=None)
  balanced_accuracy = balanced_accuracy_score(y_true, y_pred)
  auc = roc_auc_score(y_true, y_pred, average=None)
  metrics_df = pd.DataFrame([[precision_0, precision_1, recall_0, recall_1, auc, balanced_accuracy]], columns=["precision_0", "precision_1", "recall_0", "recall_1", "auc", "balanced_accuracy"])
  return metrics_df

# Load pretrained model and do prediction


In [ ]:
pretrained_model = tf.keras.models.load_model('/content/keras_model.h5')

In [ ]:
# Do prediction on the test data

prediction_result = do_prediction(test_data_embedding, pretrained_model)

In [ ]:
# Score the model performance over the test data

score_result = model_scoring(prediction_result)
score_result